# Pandas II: Data Wrangling Activity

- Use dataset: `retail_product_dataset.csv`

In [180]:
import pandas as pd

## Data Loading and Inspection

In [3]:
retail = pd.read_csv('retail_product_dataset.csv')
retail.head()

,Category,Price,Rating,Stock,Discount
0,NaN,5548.0,1.870322,NaN,0.0
1,NaN,3045.0,4.757798,NaN,38.0
2,NaN,4004.0,NaN,In Stock,0.0
3,NaN,4808.0,1.492085,NaN,33.0
4,NaN,1817.0,NaN,Out of Stock,23.0


## Summary stats & quick counts 

1. Show the mean, median, and standard deviation for **Price** and **Rating**.

2. What are the min and max values for **Discount**?

3. How many rows per **Category**? Return a Series sorted descending.


In [141]:
df = retail

In [5]:
num_stats = retail[['Price', 'Rating']].agg(['mean','median','std'])
num_stats

,Price,Rating
mean,5016.970630,3.038293
median,4996.500000,3.082060
std,2839.984813,1.143074


In [6]:
min_max = retail['Discount'].agg(['min', 'max'])
min_max

min     0.0
max    49.0
Name: Discount, dtype: float64

In [7]:
retail['Category'].value_counts().sort_values()

Category
B    378
A    403
D    408
C    425
Name: count, dtype: int64

## Handling nulls carefully

1. Create a copy `df2` to work on.

2. Fill missing **Category** with the string `'Unknown'`.

3. Fill missing **Discount** with `0`.

4. Fill missing **Stock** with `'Unknown'`.

5. For **Rating**, fill missing values with the **mean Rating per Category** (and if Category is `'Unknown'`, use the overall mean).

6. Drop rows where **Price** is missing.


In [159]:
df2 = retail.copy()

In [160]:
df2['Category'] = df2['Category'].fillna('Unknown')
df2['Stock'] = df2['Stock'].fillna('Unknown')
df2['Discount'] = df2['Discount'].fillna(0)

df2.isna().sum()

Category       0
Price        174
Rating      2050
Stock          0
Discount       0
dtype: int64

In [189]:
overall_mean = df2.Rating.mean()
category_mean = df2.groupby("Category")["Rating"].mean()

df2['Rating'] = df2['Rating'].fillna(category_mean)

df2.head()

,Category,Price,Rating,Stock,Discount,Good Deals,NetPrice,PriceBand,SaleDate,Year,Month,Average
0,Unknown,5548.0,1.870322,None,0.0,False,5548.00,200+,2024-01-01,2024,1,3819.13956
1,Unknown,3045.0,4.757798,None,38.0,False,1887.90,200+,2024-01-02,2024,1,3819.13956
3,Unknown,4808.0,1.492085,None,33.0,False,3221.36,200+,2024-01-03,2024,1,3819.13956
6,C,667.0,3.668341,True,41.0,True,393.53,200+,2024-01-04,2024,1,3819.13956
7,A,7125.0,4.983998,False,7.0,False,6626.25,200+,2024-01-05,2024,1,3819.13956


In [22]:
df2 = df2.dropna(subset=['Price']).copy()

df2.isna().sum()

Category       0
Price          0
Rating      1963
Stock          0
Discount       0
dtype: int64

## Selecting Series, subsetting columns, and masks

1. Select the **Price** Series and show the top 5 values.

2. Create a DataFrame `df_small` with columns `['Category', 'Price', 'Rating', 'Discount']`.

3. Select only numeric columns from `df2` using dtypes.

4. Create a boolean mask for **good deals**: `Stock == 'In Stock'` and `Discount >= 30` and `Price >= 50`. Use it to filter `df2`.

In [162]:
df2['Price'].head()

df_small = df2[['Category', 'Price', 'Rating', 'Discount']]

df2.describe()

masked = (df2['Stock']=='In Stock') & (df2['Discount'] >=30) & (df2['Price'] >= 50)

df2['Good Deals'] = masked

df2.head()

,Category,Price,Rating,Stock,Discount,Good Deals
0,Unknown,5548.0,1.870322,Unknown,0.0,False
1,Unknown,3045.0,4.757798,Unknown,38.0,False
2,Unknown,4004.0,NaN,In Stock,0.0,False
3,Unknown,4808.0,1.492085,Unknown,33.0,False
4,Unknown,1817.0,NaN,Out of Stock,23.0,False


## Aggregations & GroupBy

1. Add a new column **NetPrice** = `Price * (1 - Discount/100)`.

2. Compute, per **Category**, the **count of rows**, the **average NetPrice**, and the **average Rating** (after your null handling)

3. Sort the result by **average NetPrice** descending.

In [163]:
net_price = df2['Price'] * (1 - df2['Discount']/100)

df2['NetPrice'] = net_price

df2 = df2.dropna(axis=0)

compute = df2.groupby('Category').agg(RowCount=('Category', 'count'),AverageNetPrice=('NetPrice','mean'),AverageRating=('Rating', 'mean'))

compute.sort_values('AverageNetPrice', ascending = False)

,RowCount,AverageNetPrice,AverageRating
Category,,,
B,200,4080.403850,3.089855
D,219,4077.676073,3.104342
C,217,3912.770092,2.886018
Unknown,1387,3885.279056,3.038300
A,202,3786.312871,3.084678


## Apply / Map for tidy columns

1. Map **Stock** to a boolean column **InStock**: `'In Stock' -> True`, `'Out of Stock' -> False`, `'Unknown' -> None`.

2. Create a column **PriceBand** using `.apply` + `lambda`:

   - `'<50'` if Price < 50

   - `'50-199'` if 50 ≤ Price < 200

   - `'200+'` otherwise

3. Show `value_counts()` for **PriceBand**.


In [233]:
df2['Stock'] = df2['Stock'].apply(lambda s: 'True' if s == 'In Stock' else ('False' if s == 'Out of Stock' else 'None'))

df2['PriceBand'] = df2['Price'].apply(lambda p: '<50' if p < 50 else ('50 - 199' if p < 200 else '200+'))

#df2.value_counts('PriceBand')

,Category,Price,Rating,Stock,Discount,Good Deals,NetPrice,PriceBand,SaleDate,Year,Month,Average
0,Unknown,5548.0,1.870322,None,0.0,False,5548.00,200+,2024-01-01,2024,1,3819.13956
1,Unknown,3045.0,4.757798,None,38.0,False,1887.90,200+,2024-01-02,2024,1,3819.13956
3,Unknown,4808.0,1.492085,None,33.0,False,3221.36,200+,2024-01-03,2024,1,3819.13956
6,C,667.0,3.668341,None,41.0,True,393.53,200+,2024-01-04,2024,1,3819.13956
7,A,7125.0,4.983998,None,7.0,False,6626.25,200+,2024-01-05,2024,1,3819.13956


## Working with dates & masks

1. Create a date range starting on `'2024-01-01'` with **length equal to `len(df2)`** and daily frequency. Assign it to a new column **SaleDate**.

2. Parse **SaleDate** to datetime (if not already) using `pd.to_datetime`.

3. Extract **Year** and **Month** into new columns.

4. Filter rows where **Year == 2024** and **Month in [1, 2, 3]** (Q1 2024). Compute the **average NetPrice** in this subset.



In [178]:
sales_date = pd.date_range(start = "2024-01-01" ,periods= len(df2), freq="D")
df2['SaleDate'] = sales_date

df2['Year'] = df2['SaleDate'].dt.year
df2['Month'] = df2['SaleDate'].dt.month

q1_2024 = df2[(df2['Year'] == 2024) & (df2['Month'].isin([1,2,3]))]
avg_net_price = q1_2024['NetPrice'].mean()

df2['Average'] = avg_net_price

df2.head()

,Category,Price,Rating,Stock,Discount,Good Deals,NetPrice,PriceBand,SaleDate,Year,Month,Average
0,Unknown,5548.0,1.870322,None,0.0,False,5548.00,200+,2024-01-01,2024,1,3819.13956
1,Unknown,3045.0,4.757798,None,38.0,False,1887.90,200+,2024-01-02,2024,1,3819.13956
3,Unknown,4808.0,1.492085,None,33.0,False,3221.36,200+,2024-01-03,2024,1,3819.13956
6,C,667.0,3.668341,True,41.0,True,393.53,200+,2024-01-04,2024,1,3819.13956
7,A,7125.0,4.983998,False,7.0,False,6626.25,200+,2024-01-05,2024,1,3819.13956


---
# Mini-Lab: Retail Snapshot Builder

**Scenario:** You’re a junior analyst. Your manager wants a quick snapshot using only the tools in scope.

**Deliverables:**

1. A **cleaned DataFrame** `df_clean` saved as `surnamename`_retail_cleaned.csv` with the following rules:

   - Fill `Category` missing -> `'Unknown'`

   - Fill `Discount` missing -> `0`

   - Fill `Stock` missing -> `'Unknown'`

   - Fill `Rating` missing -> **mean Rating per Category**; for `'Unknown'`, use overall mean

   - Drop rows with missing `Price`

   - Add `NetPrice = Price * (1 - Discount/100)`

   - Add `SaleDate` as a daily date range starting `2024-01-01` (length = number of rows)

2. A **category summary** DataFrame with these columns (sorted by `avg_netprice` desc):

   - `items` = row count

   - `avg_netprice`

   - `avg_rating`

   - `in_stock_share` = share of rows where `Stock == 'In Stock'`

3. A **Top Deals** DataFrame with rows where `Stock == 'In Stock'`, `Discount >= 30`, and `Price >= 50`, keeping columns `Category, Price, Discount, Rating, NetPrice` and sorting by `Discount` desc.

4. A **Q1 2024** summary: average `NetPrice` by **Month** (1–3) using only `dt` accessors and `groupby`.

5. Save all three DF outputs to CSV files.



In [223]:
retail = pd.read_csv('retail_product_dataset.csv')

df_clean = retail.copy()

df_clean['Category'] = df_clean['Category'].fillna('Unknown')
df_clean['Stock'] = df_clean['Stock'].fillna('Unknown')
df_clean['Discount'] = df_clean['Discount'].fillna(0)

all_mean = df_clean['Rating'].mean()
cat_mean = df_clean.groupby('Category')['Rating'].transform('mean')

df_clean['Rating'] = df_clean['Rating'].fillna(cat_mean)

df_clean = df_clean.dropna(subset = 'Price')

netprice_1 = df_clean['Price'] * (1 - df2['Discount']/100)

df_clean['Net Price'] = netprice_1

salesdate_1 = pd.date_range(start = "2024-01-01" ,periods = len(df_clean), freq="D")
df_clean['SaleDate'] = salesdate_1

df_clean.to_csv('Dela_CruzLouie_retail_cleaned.csv', index=False)

In [227]:
cat_sum = df_clean.groupby('Category').agg(items=('Category', 'count'),avg_netprice=('Net Price','mean'),avg_rating=('Rating', 'mean'),in_stock_share=('Stock', 'nunique'))
#df_clean['in_stock_share'] = df_clean['Stock'].apply(lambda s: 'In Stock' if s == 'In Stock' else 'None')

cat_sum.sort_values('avg_netprice', ascending=False)

#df_clean = df_clean.merge(cat_sum, on='Category', how='left')

,items,avg_netprice,avg_rating,in_stock_share
Category,,,,
B,369,4080.403850,3.093849,3
D,399,4077.676073,3.110004,3
C,400,3912.770092,2.882353,3
Unknown,2631,3885.279056,3.037133,3
A,389,3786.312871,3.087095,3


In [232]:
top_deals = df_clean[['Category', 'Price', 'Discount', 'Rating', 'Net Price']]
top_deals['Top Deals'] = top_deals['Price'].apply(lambda p: '<50' if p < 50 else ('50 - 199' if p < 200 else '200+'))

top_deals.head()

C:\Users\99617\AppData\Local\Temp\ipykernel_10364\4245048348.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_deals['Top Deals'] = top_deals['Price'].apply(lambda p: '<50' if p < 50 else ('50 - 199' if p < 200 else '200+'))


,Category,Price,Discount,Rating,Net Price,Top Deals
0,Unknown,5548.0,0.0,1.870322,5548.00,200+
1,Unknown,3045.0,38.0,4.757798,1887.90,200+
2,Unknown,4004.0,0.0,3.035833,NaN,200+
3,Unknown,4808.0,33.0,1.492085,3221.36,200+
4,Unknown,1817.0,23.0,3.035833,NaN,200+
